<a href="https://colab.research.google.com/github/kusasyu36/mini-rag-papers/blob/main/mini_rag_free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir -p /content/drive/MyDrive/mini_rag_papers
# !mkdir -p /content/drive/MyDrive/mini_rag_papers/data
# !mkdir -p /content/drive/MyDrive/mini_rag_papers/vectorstore

In [ ]:
import os

project_dir = "/content/drive/MyDrive/mini_rag_papers"
os.chdir(project_dir)

print("現在のディレクトリ:", os.getcwd())

現在のディレクトリ: /content/drive/MyDrive/mini_rag_papers


In [ ]:
!pip install -q sentence-transformers chromadb faiss-cpu pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 15.5 MB/s eta 

In [ ]:
import os
import pandas as pd

project_dir = "/content/drive/MyDrive/mini_rag_papers"
os.chdir(project_dir)

csv_path = "data/papers.csv"
df = pd.read_csv(csv_path)

print("論文数:", len(df))
df.head()

論文数: 30


,id,title,year,abstract,url,Unnamed: 5
0,1,Layerwised multimodal knowledge distillation f...,2024,The transformer-based model can simultaneously...,https://pubmed.ncbi.nlm.nih.gov/38569460/,NaN
1,2,Large language models and their impact in opht...,2023,The advent of generative artificial intelligen...,https://pubmed.ncbi.nlm.nih.gov/38000875/,NaN
2,3,DriveLLaVA: Human-Level Behavior Decisions via...,2024,Human-level driving is the ultimate goal of au...,https://pubmed.ncbi.nlm.nih.gov/39000891/,NaN
3,4,Enhancing the vision-language foundation model...,2024,"Recently, vision-language representation learn...",https://pubmed.ncbi.nlm.nih.gov/39146702/,NaN
4,5,Multimodal detection of hateful memes by apply...,2022,"Detrimental to individuals and society, online...",https://pubmed.ncbi.nlm.nih.gov/36095029/,NaN


In [ ]:
import pandas as pd
import os

project_dir = "/content/drive/MyDrive/mini_rag_papers"
os.chdir(project_dir)

df = pd.read_csv("data/papers.csv")

# 余計なカラムを削除（存在したら消す）
if "Unnamed: 5" in df.columns:
    df = df.drop(columns=["Unnamed: 5"])

# 必要なカラムだけにそろえる（念のため）
df = df[["id", "title", "year", "abstract", "url"]]

# id は文字列にしておくとChromaで扱いやすい
df["id"] = df["id"].astype(str)

print("列:", df.columns.tolist())
print("件数:", len(df))
df.head(3)

列: ['id', 'title', 'year', 'abstract', 'url']
件数: 30


,id,title,year,abstract,url
0,1,Layerwised multimodal knowledge distillation f...,2024,The transformer-based model can simultaneously...,https://pubmed.ncbi.nlm.nih.gov/38569460/
1,2,Large language models and their impact in opht...,2023,The advent of generative artificial intelligen...,https://pubmed.ncbi.nlm.nih.gov/38000875/
2,3,DriveLLaVA: Human-Level Behavior Decisions via...,2024,Human-level driving is the ultimate goal of au...,https://pubmed.ncbi.nlm.nih.gov/39000891/


ステップ6：無料埋め込みモデルの用意

sentence-transformers を使って、各論文の タイトル+abstract をベクトル化します。

In [ ]:
from sentence_transformers import SentenceTransformer

# 軽くてよく使われる汎用モデル（無料）
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(model_name)

print("埋め込みモデルの読み込み完了:", model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

埋め込みモデルの読み込み完了: sentence-transformers/all-MiniLM-L6-v2


ステップ7：ChromaでベクタDBを作成・登録

7-1. Chromaクライアントとコレクション作成

In [ ]:
import chromadb
from chromadb.config import Settings

# Drive上の vectorstore フォルダに永続保存
chroma_path = "vectorstore"

client = chromadb.PersistentClient(path=chroma_path)

collection = client.get_or_create_collection(
    name="papers",
)

print("コレクション名:", collection.name)

コレクション名: papers


7-2. 文書テキスト＆メタデータの準備

RAGでは「文書本体」と「メタデータ」を分けて扱うのが定番です。
	•	文書本体：タイトル＋アブストラクト
	•	メタデータ：タイトル・year・url など

In [ ]:
# 「Title: ... \n Abstract: ...」という形のテキストにする
documents = df.apply(
    lambda row: f"Title: {row['title']}\n\nAbstract: {row['abstract']}",
    axis=1
).tolist()

ids = df["id"].tolist()

# メタデータは dict のリストに
metadatas = df[["title", "year", "url"]].to_dict(orient="records")

print("文書数:", len(documents))
print("メタデータ例:", metadatas[0])

文書数: 30
メタデータ例: {'title': 'Layerwised multimodal knowledge distillation for vision-language pretrained model', 'year': 2024, 'url': 'https://pubmed.ncbi.nlm.nih.gov/38569460/'}


7-3. 埋め込みを計算して、コレクションに追加

In [ ]:
# 30件なので一気にencodeしてOK
embeddings = embed_model.encode(
    documents,
    show_progress_bar=True
)

# numpy → list（Chromaが受け取りやすい形式に）
embeddings = embeddings.tolist()

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=embeddings,
)

print("登録完了。コレクション内の件数:", collection.count())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

登録完了。コレクション内の件数: 30


ステップ8：質問から類似論文を取り出す（Retrieval 関数）

8-1. 関数を定義する

In [ ]:
import numpy as np

def retrieve_papers(query: str, k: int = 5):
    """
    query: ユーザーの質問文（英語でも日本語でもOK）
    k    : 何件の論文を取ってくるか（トップk）

    戻り値: 論文情報のリスト（各要素は dict）
    """
    # 1. 質問文を埋め込みベクトルに変換
    query_embedding = embed_model.encode([query])[0].tolist()

    # 2. Chroma に「このベクトルに近い文書をk件ください」と問い合わせ
    result = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
    )

    # 3. 結果を扱いやすい形に整える
    hits = []
    ids = result["ids"][0]
    docs = result["documents"][0]
    metas = result["metadatas"][0]
    dists = result["distances"][0]  # 数値が小さいほど“近い”

    for i in range(len(ids)):
        hits.append({
            "rank": i + 1,
            "id": ids[i],
            "distance": float(dists[i]),
            "title": metas[i].get("title"),
            "year": metas[i].get("year"),
            "url": metas[i].get("url"),
            "document": docs[i],  # Title + Abstract 全文
        })

    return hits

8-2. 使ってみる

In [ ]:
query = "vision-language foundation model in medical imaging"
hits = retrieve_papers(query, k=5)

for h in hits:
    print(f"Rank {h['rank']}  dist={h['distance']:.3f}")
    print(f"Title: {h['title']} ({h['year']})")
    print(f"URL:   {h['url']}")
    print()

Rank 1  dist=0.889
Title: Vision-language model performance on the Japanese Nuclear Medicine Board Examination: high accuracy in text but challenges with image interpretation (2025)
URL:   https://pubmed.ncbi.nlm.nih.gov/40663225/

Rank 2  dist=0.894
Title: Enhancing the vision-language foundation model with key semantic knowledge-emphasized report refinement (2024)
URL:   https://pubmed.ncbi.nlm.nih.gov/39146702/

Rank 3  dist=0.994
Title: 
A vision-language foundation model for the generation of realistic chest X-ray images (2025)
URL:   https://pubmed.ncbi.nlm.nih.gov/39187663/

Rank 4  dist=0.999
Title: Enhancing diagnostic accuracy in rare and common fundus diseases with a knowledge-rich vision-language model (2025)
URL:   https://pubmed.ncbi.nlm.nih.gov/40592857/

Rank 5  dist=1.017
Title: Diagnostic accuracy of vision-language models on Japanese diagnostic radiology, nuclear medicine, and interventional radiology specialty board examinations (2024)
URL:   https://pubmed.ncbi.nlm

ステップ9：LLM に渡すプロンプトを自動生成する（Gにつなぐ）

次は、

	1.	retrieve_papers() で取ってきた文献を

	2.	きれいに並べたテキストにまとめて
  
	3.	ChatGPT / Gemini にコピペする

ための関数を作ります。

9-1. プロンプトを組み立てる関数


In [ ]:
def build_llm_prompt(query: str, hits):
    """
    query: ユーザーの質問
    hits : retrieve_papers() の結果（論文リスト）

    戻り値: LLM にそのままコピペできるプロンプト文字列
    """
    context_blocks = []

    for h in hits:
        block = (
            f"[文献{h['rank']}]\n"
            f"Title: {h['title']}\n"
            f"Year:  {h['year']}\n"
            f"URL:   {h['url']}\n\n"
            f"{h['document']}\n"
        )
        context_blocks.append(block)

    context_text = "\n".join(context_blocks)

    prompt = f"""
あなたは論文サーベイを手伝うAI研究アシスタントです。
以下の「文献[1..{len(hits)}]」の内容だけを根拠にして、ユーザーの質問に日本語で答えてください。

・必ず根拠となる文献の内容に基づいて回答すること
・憶測で書かず、「この文献からは分からない」ことはその旨を述べること
・最後に、どの文献番号を主に参照したかを明記すること（例: 参考文献: [文献1], [文献3]）

ユーザーの質問:
{query}

--- ここから文献 ---
{context_text}
--- 文献ここまで ---
"""
    return prompt.strip()

In [ ]:
query = "これらの論文に基づいて、vision-language foundation model の最近の研究動向を簡単にまとめてください。"

hits = retrieve_papers(query, k=5)
prompt = build_llm_prompt(query, hits)

print(prompt[:2000])  # 長いので先頭だけ表示

あなたは論文サーベイを手伝うAI研究アシスタントです。
以下の「文献[1..5]」の内容だけを根拠にして、ユーザーの質問に日本語で答えてください。

・必ず根拠となる文献の内容に基づいて回答すること
・憶測で書かず、「この文献からは分からない」ことはその旨を述べること
・最後に、どの文献番号を主に参照したかを明記すること（例: 参考文献: [文献1], [文献3]）

ユーザーの質問:
これらの論文に基づいて、vision-language foundation model の最近の研究動向を簡単にまとめてください。

--- ここから文献 ---
[文献1]
Title: Vision-language model performance on the Japanese Nuclear Medicine Board Examination: high accuracy in text but challenges with image interpretation
Year:  2025
URL:   https://pubmed.ncbi.nlm.nih.gov/40663225/

Title: Vision-language model performance on the Japanese Nuclear Medicine Board Examination: high accuracy in text but challenges with image interpretation

Abstract: Objective: Vision language models (VLMs) allow visual input to Large Language Models. VLMs have been developing rapidly, and their accuracy is improving rapidly. Their performance in nuclear medicine compared to state-of-the-art models, including reasoning models, is not yet clear. We evaluated state-of-the-art VLMs using problem

In [ ]:
!pip install -q google-generativeai

In [ ]:
import os
import google.generativeai as genai

API_KEY = ""  # 後でちゃんと差し替え
genai.configure(api_key=API_KEY)

In [ ]:
from google.generativeai import GenerativeModel

model = GenerativeModel("gemini-2.5-flash")  # 無料枠向けの軽めモデル

query = "これらの論文に基づいて、vision-language foundation model の最近の研究動向を簡単にまとめてください。"

hits = retrieve_papers(query, k=5)
prompt = build_llm_prompt(query, hits)

response = model.generate_content(prompt)
print(response.text)

これらの論文に基づくと、vision-language foundation model（VLM）の最近の研究動向は以下の点が挙げられます。

1.  **性能の急速な向上と医療分野での応用**
    VLMは急速に発展しており、その精度は急速に向上しています。特に医療分野における応用が進んでおり、日本の核医学専門医試験や診断放射線学、核医学、IVRの専門医試験などにおいて、その診断精度が評価されています。[文献1, 2] 最新のモデルは、テキストベースの質問に対して高い精度を示す一方で、画像解釈を伴う質問では依然として課題があるものの、その性能は向上しつつあります。[文献1] 例えば、GPT-4oは、複数の日本の専門医試験において、画像を含む質問と全ての質問の両方で他のVLMを上回る最も高い精度を示しました。[文献2] しかし、VLMは医療分野におけるテキストベースの質問の良いアシスタントにはなり得るものの、画像を伴う包括的な質問においては限界があり、専門家の包括的な訓練や解釈を代替するものではないと結論付けられています。[文献1]

2.  **効率化と軽量化への取り組み**
    TransformerベースのVLMはマルチモーダルなアプリケーションで優れた性能を発揮しますが、大規模なパラメータ数がリソース制約のあるデバイスへの展開を妨げるという課題があります。[文献4, 5] この課題に対処するため、モデル圧縮や知識蒸留に関する研究が進められています。例えば、「Layerwised multimodal knowledge distillation」では、教師モデルの中間層も知識転移に利用し、モダリティ間の干渉を避けることで、VLMの性能を維持しつつ効率的な蒸留を実現しています。[文献4]
    さらに、計算リソースの制約を緩和するために、軽量なVLM（multimodal small language model: MSLM）の開発も重要な動向です。「SigPhi-Med」という軽量な生体医用VLMは、既存の大規模なマルチモーダル言語モデル（MLLM）のLLM部分を小型言語モデル（SLM）に置き換えることで、パラメータ数を大幅に削減し、医療タスクでの高性能を維持することを目指しています。[文献5] SigPhi-Medは、わずか4.2Bのパラメータ数